In [1]:
import json
import pandas as pd
from pathlib import Path

from datasets import load_dataset_builder, load_dataset

In [2]:
dataset_sentiment_train = load_dataset("sepidmnorozy/Norwegian_sentiment", split="train")
dataset_sentiment_test = load_dataset("sepidmnorozy/Norwegian_sentiment", split="test")
dataset_sentiment_dev = load_dataset("sepidmnorozy/Norwegian_sentiment", split="validation")

pd_dataset_train = pd.DataFrame(dataset_sentiment_train)
pd_dataset_test = pd.DataFrame(dataset_sentiment_test)
pd_dataset_dev = pd.DataFrame(dataset_sentiment_dev)

dataset_list = [pd_dataset_train, pd_dataset_test, pd_dataset_dev]
pd_whole_dataset = pd.concat(dataset_list)

pd_dataset_train.rename(columns = {'label':'rating', 'text':'excerpt'}, inplace = True)
pd_dataset_test.rename(columns = {'label':'rating', 'text':'excerpt'}, inplace = True)
pd_dataset_dev.rename(columns = {'label':'rating', 'text':'excerpt'}, inplace = True)
pd_whole_dataset.rename(columns = {'label':'rating', 'text':'excerpt'}, inplace = True)

pd_whole_dataset

Using custom data configuration sepidmnorozy--Norwegian_sentiment-45f1d0d212138cf3
Found cached dataset csv (C:/Users/marcu/.cache/huggingface/datasets/sepidmnorozy___csv/sepidmnorozy--Norwegian_sentiment-45f1d0d212138cf3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration sepidmnorozy--Norwegian_sentiment-45f1d0d212138cf3
Found cached dataset csv (C:/Users/marcu/.cache/huggingface/datasets/sepidmnorozy___csv/sepidmnorozy--Norwegian_sentiment-45f1d0d212138cf3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration sepidmnorozy--Norwegian_sentiment-45f1d0d212138cf3
Found cached dataset csv (C:/Users/marcu/.cache/huggingface/datasets/sepidmnorozy___csv/sepidmnorozy--Norwegian_sentiment-45f1d0d212138cf3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


,rating,excerpt
0,0,Verre er det med slagsmålene .
1,0,"Klipperen har overtatt Seagals martial art , o..."
2,1,Ei finslig dame skal bli med på kjøret og obse...
3,0,"Hvis du vil fremstå som seriøs filmskaper , ik..."
4,0,Ingenting sier studentfilm som dét .
...,...,...
511,1,Det er flere som har det tungt etter Gunvalds ...
512,1,En frisk erstatter for Mikael Persbrandt
513,1,"God til å lese folk , god til å jatte med folk..."
514,1,Kjemien mellom Hovland og Martin Beck er klede...


In [3]:
file = open('review_data/orig_data/reviews.json', encoding="utf8")
data = json.load(file)

print(len(data))

train_list = []
test_list = []
dev_list = []

for k, v in data.items():
    
    excerpt, title, rating, split, category, language, id, year = None, None, None, None, None, None, None, None
    
    for k2, v2 in v.items():
        
        if k2 == "excerpt":
            excerpt = None if v2 == "" else v2
        elif k2 == "title":
            title = v2
        elif k2 == "rating":
            rating = None if v2 == 4 else v2
        elif k2 == "split":
            split = v2
        elif k2 == "category":
            category = v2
        elif k2 == "language":
            language = v2
        elif k2 == "id":
            id = v2
        elif k2 == "year":
            year = v2
        
    if [x for x in (id, year, category, language, title, excerpt, rating) if x is None]:
        continue
    elif split == "train":
        train_list.append((split, id, year, category, language, title, excerpt, 0 if rating <= 3 else 1))
    elif split == "test":
        test_list.append((split, id, year, category, language, title, excerpt, 0 if rating <= 3 else 1))
    elif split == "dev":
        dev_list.append((split, id, year, category, language, title, excerpt, 0 if rating <= 3 else 1))

file.close()

print(f"Train length: {len(train_list)} | Test length: {len(test_list)} | Dev length: {len(dev_list)} \n")

train_df = pd.DataFrame(train_list, columns=['split', 'review_id', 'year', 'category', 'language', 'title', 'excerpt', 'rating'])
test_df = pd.DataFrame(test_list, columns=['split', 'review_id', 'year', 'category', 'language', 'title', 'excerpt', 'rating'])
dev_df = pd.DataFrame(dev_list, columns=['split', 'review_id', 'year', 'category', 'language', 'title', 'excerpt', 'rating'])

df_list = [train_df, test_df, dev_df]
all_dfs = pd.concat(df_list)

all_dfs

43614
Train length: 20679 | Test length: 2584 | Dev length: 2512 



,split,review_id,year,category,language,title,excerpt,rating
0,train,0,2007,screen,nb,Rome S02,Toppen innen tv-drama akkurat nå!,1
1,train,1,2007,screen,nb,Twin Peaks - definitive gold box edition,Gull for <em>Twin Peaks</em>-fans!,1
2,train,2,2008,screen,nb,The Wire (sesong 1-4),"The Wire vil gjøre deg avhengig, men på en god...",1
3,train,3,2008,screen,nb,Mad Men (sesong 1),"Stilig, underholdende og sofistikert!",1
4,train,4,2009,screen,nb,Mad Men (sesong 2),TV-underholdning av høyeste kvalitet!,1
...,...,...,...,...,...,...,...,...
2507,dev,706854,2019,misc,nb,«Klarsynt»-portrett med åpenbare mangler,FILMANMELDELSE: Nesten fin film om det alterna...,0
2508,dev,706855,2019,misc,nb,Klassisk soul med sterke harmonier,Fem unge herrer med forbilder som Jackie Wilso...,1
2509,dev,706856,2019,misc,nb,Myk americana om vonde tider,"Sorg kan følge glede, men troen tar ingen fra ...",1
2510,dev,706859,2019,misc,nb,Sjelden vare i flott debut,BOK: En feel good-roman som faktisk makter å f...,1


In [4]:
concated_train = pd.concat([train_df, pd_dataset_train], axis=0, ignore_index=True)
concated_test = pd.concat([test_df, pd_dataset_test], axis=0, ignore_index=True)
concated_dev = pd.concat([dev_df, pd_dataset_dev], axis=0, ignore_index=True)

all_dfs_concated = pd.concat([concated_train, concated_test, concated_dev])
all_dfs_concated

,split,review_id,year,category,language,title,excerpt,rating
0,train,0.0,2007.0,screen,nb,Rome S02,Toppen innen tv-drama akkurat nå!,1
1,train,1.0,2007.0,screen,nb,Twin Peaks - definitive gold box edition,Gull for <em>Twin Peaks</em>-fans!,1
2,train,2.0,2008.0,screen,nb,The Wire (sesong 1-4),"The Wire vil gjøre deg avhengig, men på en god...",1
3,train,3.0,2008.0,screen,nb,Mad Men (sesong 1),"Stilig, underholdende og sofistikert!",1
4,train,4.0,2009.0,screen,nb,Mad Men (sesong 2),TV-underholdning av høyeste kvalitet!,1
...,...,...,...,...,...,...,...,...
3023,NaN,NaN,NaN,NaN,NaN,NaN,Det er flere som har det tungt etter Gunvalds ...,1
3024,NaN,NaN,NaN,NaN,NaN,NaN,En frisk erstatter for Mikael Persbrandt,1
3025,NaN,NaN,NaN,NaN,NaN,NaN,"God til å lese folk , god til å jatte med folk...",1
3026,NaN,NaN,NaN,NaN,NaN,NaN,Kjemien mellom Hovland og Martin Beck er klede...,1


In [6]:
all_dfs_concated["rating"].value_counts()

1    19140
0    10243
Name: rating, dtype: int64

In [8]:
concated_train.to_csv('review_data/binary_not_4_concat/train_reviews_binary_concat.csv', index=False)
concated_test.to_csv('review_data/binary_not_4_concat/test_reviews_binary_concat.csv', index=False)
concated_dev.to_csv('review_data/binary_not_4_concat/dev_reviews_binary_concat.csv', index=False)
all_dfs_concated.to_csv('review_data/binary_not_4_concat/all_reviews_binary_concat.csv', index=False)